In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import chess.pgn as pgn
import chess
import numpy as np
from math import ceil
from sklearn.feature_extraction import DictVectorizer
from collections import defaultdict, Counter

In [2]:
data=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_games.csv')
data.Scores=data.Scores.str.replace(' ','').str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Scores=data.Scores.apply(lambda x: list(map(int,x.split())))
data.Scores=data.Scores.apply(lambda x: list(map(int,x)))
data.Moves=data.Moves.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Moves=data.Moves.apply(lambda x: x.split())
data.UCI=data.UCI.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.UCI=data.UCI.apply(lambda x: x.split())
data.head()

,Result,Moves,UCI,Scores,NumMoves,MeanScore,ModeScore,FinalScore
0,1/2-1/2,"[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...","[g1f3, g8f6, c2c4, c7c5, b2b3, g7g6, c1b2, f8g...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...",19,23.947368,3,54
1,0-1,"[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...","[e2e4, d7d5, e4d5, g8f6, d2d4, f6d5, g1f3, g7g...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...",53,-692.198113,18,-11544
2,1-0,"[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...","[c2c4, g8f6, b1c3, d7d6, d2d4, e7e5, g1f3, b8d...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...",39,49.246753,31,196
3,1-0,"[e4, c5, Nf3, d6, b4, Nf6, bxc5, Nxe4, cxd6, Q...","[e2e4, c7c5, g1f3, d7d6, b2b4, g8f6, b4c5, f6e...","[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9...",25,306.040816,18,875
4,1/2-1/2,"[Nf3, d5, d4, Nf6, c4, e6, Nc3, Be7, Bg5, h6, ...","[g1f3, d7d5, d2d4, g8f6, c2c4, e7e6, b1c3, f8e...","[18, 29, 10, 22, 20, 18, 19, 28, 24, 18, 17, 1...",29,30.827586,18,-11


In [3]:
elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,57,4765
1,1915,1999,1957,84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [4]:
features=data.copy()

'''
# Может попробовать так? (долго думает, пока непонятно надо это или нет)
wins = pd.DataFrame(columns = ["WhiteWins", "BlackWins"])
features=features.append(wins)
features.update(features[["WhiteWins", "BlackWins"]].fillna(0, inplace=False), overwrite=False)

for i,x in enumerate(features['Result']):
    if (x == "1/2-1/2"):
        features['WhiteWins'][i] = 1
        features['BlackWins'][i] = 1
    elif (x == "1-0"):
        features['WhiteWins'][i] = 1
    else:
        features['BlackWins'][i] = 1
        '''

features.Result=features.Result.apply(lambda x: 0.5 if x=='1/2-1/2' else x)
features.Result=features.Result.apply(lambda x: 1 if x=='1-0' else x)
features.Result=features.Result.apply(lambda x: 0 if x=='0-1' else x)

features['FullMoves']=features.Moves.apply(lambda x: len(x))

features=features.drop(columns=['Moves','Scores','UCI'])
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves
0,0.5,19,23.947368,3,54,38
1,0.0,53,-692.198113,18,-11544,106
2,1.0,39,49.246753,31,196,77
3,1.0,25,306.040816,18,875,49
4,0.5,29,30.827586,18,-11,58


In [5]:
data['WhiteMoves']=data.Moves.apply(lambda x: x[::2])
data['WhiteMovesUCI']=data.UCI.apply(lambda x: x[::2])
data['WhiteScores']=data.Scores.apply(lambda x: x[::2])
data['BlackMoves']=data.Moves.apply(lambda x: x[1::2])
data['BlackMovesUCI']=data.UCI.apply(lambda x: x[1::2])
data['BlackScores']=data.Scores.apply(lambda x: x[1::2])
data['DiffScores']=data.Scores.apply(lambda x: np.diff(x).tolist())
data['DiffWhiteScores']=data.DiffScores.apply(lambda x: x[::2])
data['DiffBlackScores']=data.DiffScores.apply(lambda x: x[1::2])

data=data.drop(columns=['NumMoves','MeanScore','ModeScore','FinalScore'])
data.head(3)

,Result,Moves,UCI,Scores,WhiteMoves,WhiteMovesUCI,WhiteScores,BlackMoves,BlackMovesUCI,BlackScores,DiffScores,DiffWhiteScores,DiffBlackScores
0,1/2-1/2,"[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...","[g1f3, g8f6, c2c4, c7c5, b2b3, g7g6, c1b2, f8g...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...","[Nf3, c4, b3, Bb2, e3, Be2, O-O, Nc3, Qc2, Rac...","[g1f3, c2c4, b2b3, c1b2, e2e3, f1e2, e1g1, b1c...","[18, 12, -5, 3, 22, 20, 8, 11, -6, 1, -21, -26...","[Nf6, c5, g6, Bg7, O-O, b6, Bb7, Nc6, Rc8, d5,...","[g8f6, c7c5, g7g6, f8g7, e8g8, b7b6, c8b7, b8c...","[17, 8, 12, -2, 21, 13, 21, 3, 5, -10, -1, 18,...","[-1, -5, -4, -13, 17, -9, -5, 24, -1, -1, -7, ...","[-1, -4, 17, -5, -1, -7, 13, -8, 11, -11, 20, ...","[-5, -13, -9, 24, -1, -5, -10, -9, -4, -11, -2..."
1,0-1,"[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...","[e2e4, d7d5, e4d5, g8f6, d2d4, f6d5, g1f3, g7g...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...","[e4, exd5, d4, Nf3, Be2, c4, Nc3, O-O, Be3, d5...","[e2e4, e4d5, d2d4, g1f3, f1e2, c2c4, b1c3, e1g...","[26, 68, 65, 48, 61, 63, 53, 69, 30, -2, -11, ...","[d5, Nf6, Nxd5, g6, Bg7, Nb6, O-O, Nc6, Bg4, B...","[d7d5, g8f6, f6d5, g7g6, f8g7, d5b6, e8g8, b8c...","[51, 57, 77, 93, 63, 58, 46, 29, 27, 12, 0, 24...","[25, 17, -11, 8, 12, -29, 45, -32, 2, 0, -5, -...","[25, -11, 12, 45, 2, -5, -7, -40, -3, 14, 11, ...","[17, 8, -29, -32, 0, -5, 23, 1, -29, -23, -17,..."
2,1-0,"[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...","[c2c4, g8f6, b1c3, d7d6, d2d4, e7e5, g1f3, b8d...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...","[c4, Nc3, d4, Nf3, Bg5, e3, Qc2, Bd3, h4, dxe5...","[c2c4, b1c3, d2d4, g1f3, c1g5, e2e3, d1c2, f1d...","[2, 5, 35, 37, 10, 8, 30, 13, -12, 13, -22, -1...","[Nf6, d6, e5, Nbd7, Be7, c6, O-O, h6, b5, dxe5...","[g8f6, d7d6, e7e5, b8d7, f8e7, c7c6, e8g8, h7h...","[21, 53, 45, 54, 22, 48, 17, 35, 31, 31, 20, -...","[19, -16, 48, -18, 10, -8, 17, -44, 12, -14, 4...","[19, 48, 10, 17, 12, 40, -13, 22, 43, 18, 42, ...","[-16, -18, -8, -44, -14, -18, -4, -47, -18, -5..."


#### Условные разделения на дебют, миттельшпиль и эндшпиль.
##### (1-15 - дебют, 16-35 - миттельшпиль, 36+ эндшпиль)
Разделения очень-очень условные, но имеют основания, так как игроки могут по-разному показывать себя в разных частях партии.

In [6]:
data = data.append(pd.DataFrame(columns = ["DebScores", "MitScores", "EndScores"]))

data['DebScores']=data.Scores.apply(lambda x: x[:16])
data['MitScores']=data.Scores.apply(lambda x: x[16:36])
data['EndScores']=data.Scores.apply(lambda x: x[36:])


modes=['DebScores','MitScores','EndScores']
var=['White','Black','Diff','DiffWhite','DiffBlack']

for m in modes:
    data['White' + m]=data[m].apply(lambda x: x[::2])
    data['Black' + m]=data[m].apply(lambda x: x[1::2])
    data['Diff'+m]=data[m].apply(lambda x: np.diff(x).tolist())
    data['DiffWhite'+m]=data['Diff'+m].apply(lambda x: x[::2])
    data['DiffBlack'+m]=data['Diff'+m].apply(lambda x: x[1::2])


In [7]:
data.head()

,BlackMoves,BlackMovesUCI,BlackScores,DebScores,DiffBlackScores,DiffScores,DiffWhiteScores,EndScores,MitScores,Moves,...,WhiteMitScores,BlackMitScores,DiffMitScores,DiffWhiteMitScores,DiffBlackMitScores,WhiteEndScores,BlackEndScores,DiffEndScores,DiffWhiteEndScores,DiffBlackEndScores
0,"[Nf6, c5, g6, Bg7, O-O, b6, Bb7, Nc6, Rc8, d5,...","[g8f6, c7c5, g7g6, f8g7, e8g8, b7b6, c8b7, b8c...","[17, 8, 12, -2, 21, 13, 21, 3, 5, -10, -1, 18,...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...","[-5, -13, -9, 24, -1, -5, -10, -9, -4, -11, -2...","[-1, -5, -4, -13, 17, -9, -5, 24, -1, -1, -7, ...","[-1, -4, 17, -5, -1, -7, 13, -8, 11, -11, 20, ...","[35, 54]","[-6, 5, 1, -10, -21, -1, -26, 18, 48, 48, 53, ...","[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...",...,"[-6, 1, -21, -26, 48, 53, 46, 51, 54, 70]","[5, -10, -1, 18, 48, 73, 68, 60, 46, 62]","[11, -4, -11, -11, 20, -25, 44, 30, 0, 5, 20, ...","[11, -11, 20, 44, 0, 20, 22, 9, -8, -8]","[-4, -11, -25, 30, 5, -27, -17, -6, 24]",[35],[54],[19],[19],[]
1,"[d5, Nf6, Nxd5, g6, Bg7, Nb6, O-O, Nc6, Bg4, B...","[d7d5, g8f6, f6d5, g7g6, f8g7, d5b6, e8g8, b8c...","[51, 57, 77, 93, 63, 58, 46, 29, 27, 12, 0, 24...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...","[17, 8, -29, -32, 0, -5, 23, 1, -29, -23, -17,...","[25, 17, -11, 8, 12, -29, 45, -32, 2, 0, -5, -...","[25, -11, 12, 45, 2, -5, -7, -40, -3, 14, 11, ...","[18, 20, 10, 2, 7, 22, -28, -26, -30, -34, -29...","[30, 27, -2, 12, -11, 0, -17, 24, 5, 15, 16, 3...","[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...",...,"[30, -2, -11, -17, 5, 16, 44, 18, 27, 18]","[27, 12, 0, 24, 15, 31, 25, 20, 26, 14]","[-3, -29, 14, -23, 11, -17, 41, -19, 10, 1, 15...","[-3, 14, 11, 41, 10, 15, -19, 2, -1, -4]","[-29, -23, -17, -19, 1, 13, -7, 7, -8]","[18, 10, 7, -28, -30, -29, -49, -51, -82, -83,...","[20, 2, 22, -26, -34, -28, -55, -50, -74, -77,...","[2, -10, -8, 5, 15, -50, 2, -4, -4, 5, 1, -21,...","[2, -8, 15, 2, -4, 1, -6, 1, 8, 6, 20, 6, 13, ...","[-10, 5, -50, -4, 5, -21, 4, -32, -9, -15, -33..."
2,"[Nf6, d6, e5, Nbd7, Be7, c6, O-O, h6, b5, dxe5...","[g8f6, d7d6, e7e5, b8d7, f8e7, c7c6, e8g8, h7h...","[21, 53, 45, 54, 22, 48, 17, 35, 31, 31, 20, -...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...","[-16, -18, -8, -44, -14, -18, -4, -47, -18, -5...","[19, -16, 48, -18, 10, -8, 17, -44, 12, -14, 4...","[19, 48, 10, 17, 12, 40, -13, 22, 43, 18, 42, ...","[-10, 12, -11, 0, -11, 29, 27, 31, 26, 24, 28,...","[-12, 31, 13, 31, -22, 20, -18, -30, -27, 6, -...","[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...",...,"[-12, 13, -22, -18, -27, -11, 53, 56, 68, 44]","[31, 31, 20, -30, 6, 74, 50, 55, 58, 59]","[43, -18, 18, -53, 42, -38, -12, 3, 33, -17, 8...","[43, 18, 42, -12, 33, 85, -3, -1, -10, 15]","[-18, -53, -38, 3, -17, -21, 6, 13, -14]","[-10, -11, -11, 27, 26, 28, 36, 46, 44, 40, 34...","[12, 0, 29, 31, 24, 42, 38, 36, 45, 36, 68, 53...","[22, -23, 11, -11, 40, -2, 4, -5, -2, 4, 14, -...","[22, 11, 40, 4, -2, 14, 2, -10, 1, -4, 34, -2,...","[-23, -11, -2, -5, 4, -6, 8, 8, -5, -2, -13, -..."
3,"[c5, d6, Nf6, Nxe4, Qb6, Bg4, Nxd6, Nc6, Nb4, ...","[c7c5, d7d6, g8f6, f6e4, d8b6, c8g4, e4d6, b8c...","[64, 53, 20, 20, 49, 95, 82, 93, 59, 142, 193,...","[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9...","[-29, -35, -2, -10, 11, -4, 1, -35, 39, 4, -72...","[38, -29, 18, -35, 2, -2, 2, -10, 39, 11, 35, ...","[38, 18, 2, 2, 39, 35, -9, 10, 1, 44, 47, 8, 1...","[505, 567, 525, 834, 819, 870, 848, 860, 826, ...","[58, 59, 98, 142, 146, 193, 121, 129, 135, 136...","[e4, c5, Nf3, d6, b4, Nf6, bxc5, Nxe4, cxd6, Q...",...,"[58, 98, 146, 121, 135, 140, 158, 186, 227, 500]","[59, 142, 193, 129, 136, 152, 210, 235, 521, 489]","[1, 39, 44, 4, 47, -72, 8, 6, 1, 4, 12, 6, 52,...","[1, 44, 47, 8, 1, 12, 52, 49, 294, -11]","[39, 4, -72, 6, 4, 6, -24, -8, -21]","[505, 525, 819, 848, 826, 830, 875]","[567, 834, 870, 860, 917, 868]","[62, -42, 309, -15, 51, -22, 12, -34, 91, -87,...","[62, 309, 51, 12, 91, 38]","[-42, -15, -22, -34, -87, 7]"
4,"[d5, Nf6, e6,

In [8]:
stats = [np.min, np.max, np.mean, lambda x: np.median(np.abs(x)),np.std, 
         lambda x: round(np.var(x),2), lambda x: max(set(x), key=x.count)]
stat_names = ['Min', 'Max', 'Mean', 'Median','Std','Variance','Mode']
color=['White','Black']

# общие оценки
for stat, stat_name in zip(stats, stat_names):
    features[stat_name + 'Score'] = data.Scores.apply(lambda x: stat(x))
    
# общие оценки по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + 'Score'] = data[c+'Scores'].apply(lambda x: stat(x))
        
    features['Final' +c+ 'Score']=data[c+'Scores'].apply(lambda x: x[-1])


# по разнцие в оценках в общем
for stat, stat_name in zip(stats, stat_names):
    features[stat_name + 'DeltaScore'] = data.DiffScores.apply(lambda x: stat(x))
    
# по разнцие в оценках по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + 'DeltaScore'] = data['Diff'+c+'Scores'].apply(lambda x: stat(x))
    
##########################################################################################################################    
# по частям партий (и все тоже самое, что выше)
modes=['Deb','Mit','End']

for m in modes:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + m+'Score'] = data[m+'Scores'].apply(lambda x: stat(x) if  len(x)!=0 else -1)

# по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + m+'Score'] = data[c+m+'Scores'].apply(lambda x: stat(x) if  len(x)!=0 else -1)

    features['Final' +c+ m+'Score']=data[c+m+'Scores'].apply(lambda x: x[-1]  if  len(x)!=0 else 0)

# по разнцие в оценках
for m in modes:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + m+'DeltaScore'] = data['Diff'+m+'Scores'].apply(lambda x: stat(x)  if  len(x)!=0 else -1)
        
# по разнцие в оценках по цветам
for c in color:
    for m in modes:
        for stat, stat_name in zip(stats, stat_names):
            features[stat_name + c + m+'DeltaScore'] = data['Diff'+c+m+'Scores'].apply(lambda x: stat(x)  if  len(x)!=0 else -1)    


In [9]:
print(features.shape)
features.head(3)

(24255, 148)


,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,StdBlackMitDeltaScore,VarianceBlackMitDeltaScore,ModeBlackMitDeltaScore,MinBlackEndDeltaScore,MaxBlackEndDeltaScore,MeanBlackEndDeltaScore,MedianBlackEndDeltaScore,StdBlackEndDeltaScore,VarianceBlackEndDeltaScore,ModeBlackEndDeltaScore
0,0.5,19,23.947368,3,54,38,-26,73,20.5,26.431587,...,18.886274,356.69,5,-1,-1,-1.000000,-1.0,-1.000000,-1.00,-1
1,0.0,53,-692.198113,18,-11544,106,-11544,93,63.0,2286.131735,...,13.353688,178.32,1,-9578,92,-428.235294,19.5,1643.924055,2702486.30,5
2,1.0,39,49.246753,31,196,77,-30,198,37.0,51.525093,...,19.917112,396.69,3,-31,31,-1.450000,8.0,14.623526,213.85,8


#### Индекс первого и последнего хода, когда белые или черные получили преимущество (>100 cp)
+победили ли они после получения преимущества (если не было преимущества - то сверяем с ничьей)

In [10]:
def find_advantage(scores):
    for i,x in enumerate(scores):
        if abs(x) >=100:
            return i
    return 0

def is_realised_advantage(ind,result,num=0):
    if ind == num and num != 0:
        return 1
    elif ind==0 and result==0.5:
        return 1
    elif ind!=0 and ind %2!=0 and result==1:
        return 1
    elif ind!=0 and ind %2==0 and result==0:
        return 1
    else:
        return 0        
    
    
features['FirstAdvantageInd']=data.Scores.apply(find_advantage)
features['IsFirstAdvantageRealised']=features[['FirstAdvantageInd','Result']].apply(lambda x: is_realised_advantage(*x),axis=1)
features['FirstAdvantageInd']=features['FirstAdvantageInd'].apply(lambda x: ceil(x/2))

features['LastAdvantageInd']=data.Scores.apply(lambda x: np.argmax(np.absolute(x) >=100) or len(x))
features['IsLastAdvantageRealised']=features[['LastAdvantageInd','Result','NumMoves']].apply(lambda x: is_realised_advantage(*x),axis=1)
features['LastAdvantageInd']=features['LastAdvantageInd'].apply(lambda x: ceil(x/2))

features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,MaxBlackEndDeltaScore,MeanBlackEndDeltaScore,MedianBlackEndDeltaScore,StdBlackEndDeltaScore,VarianceBlackEndDeltaScore,ModeBlackEndDeltaScore,FirstAdvantageInd,IsFirstAdvantageRealised,LastAdvantageInd,IsLastAdvantageRealised
0,0.5,19,23.947368,3,54,38,-26,73,20.5,26.431587,...,-1,-1.000000,-1.0,-1.000000,-1.00,-1,0,1,19,0
1,0.0,53,-692.198113,18,-11544,106,-11544,93,63.0,2286.131735,...,92,-428.235294,19.5,1643.924055,2702486.30,5,29,1,29,1
2,1.0,39,49.246753,31,196,77,-30,198,37.0,51.525093,...,31,-1.450000,8.0,14.623526,213.85,8,33,1,33,1
3,1.0,25,306.040816,18,875,49,10,917,142.0,313.263595,...,7,-32.166667,28.0,28.979399,839.81,7,10,1,10,1
4,0.5,29,30.827586,18,-11,58,-15,94,29.0,24.845827,...,29,-6.000000,10.5,22.013632,484.60,3,0,1,29,0


#### Кто предложил ничью?

In [17]:
# Не надо, я думаю

'''
def who_offer_draw(res,num):
    if res==0.5:
        if num % 2==0:
            return 0 # код для черных
        else:
            return 1 # код для белых
    else:
        return -1 # никто не предлагал
        
features['OfferDraw']=features[['Result','FullMoves']].apply(lambda x: who_offer_draw(*x),axis=1)
features.head()
'''

,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,VarianceBlackEndDeltaScore,ModeBlackEndDeltaScore,FirstAdvantageInd,IsFirstAdvantageRealised,LastAdvantageInd,IsLastAdvantageRealised,OfferDraw,FrequentDebut,CommonDebut,RareDebut
0,0.5,19,23.947368,3,54,38,-26,73,20.5,26.431587,...,-1.00,-1,0,1,19,0,0,0,0,1
1,0.0,53,-692.198113,18,-11544,106,-11544,93,63.0,2286.131735,...,2702486.30,5,29,1,29,1,-1,0,1,0
2,1.0,39,49.246753,31,196,77,-30,198,37.0,51.525093,...,213.85,8,33,1,33,1,-1,0,0,1
3,1.0,25,306.040816,18,875,49,10,917,142.0,313.263595,...,839.81,7,10,1,10,1,-1,0,0,1
4,0.5,29,30.827586,18,-11,58,-15,94,29.0,24.845827,...,484.60,3,0,1,29,0,0,0,1,0


#### Совершались ли "грубые" ошибки? (Изменение в оценке позиции > 100) 
+ход, на котором была совершена грубая ошибка

+поиск количества ошибок (Количество раз, когда оценка менялась более, чем на 50)

In [11]:
def find_erros(x,mode):
    count_erros=0
    count_blunders=0
    ind_first_blunder=0
    ind_first_error=0
    first_blunder=True
    first_error=True
    for i in range(len(x)-1):
        sc=abs(x[i+1]-x[i])
        if sc >=50:
            count_erros+=1
            if first_error:
                ind_first_error =i
                first_error=False
        if sc >=100:
            count_blunders+=1
            if first_blunder:
                ind_first_blunder=i
                first_blunder=False 
    if mode=='ce':
        return count_erros
    elif mode=='cb':
        return count_blunders
    elif mode=='ie':
        return ind_first_error
    elif mode=='ib':
        return ind_first_blunder

In [12]:
color=['White','Black']
modes=['Deb','Mit','End']

# были ли ошибки общие и по цветам
features['IndError'] = data.Scores.apply(find_erros,args=('ie',))
features['IndBlunder'] = data.Scores.apply(find_erros,args=('ib',))
for c in color:
    features['Ind'+c+'Error'] = data[c+'Scores'].apply(find_erros,args=('ie',))
    features['Ind'+c+'Blunder'] = data[c+'Scores'].apply(find_erros,args=('ib',))
    
# были ли ошибки общие и по цветам в разных частях партии
for m in modes:
    features['Ind'+m+'Error'] = data[m+'Scores'].apply(find_erros,args=('ie',))
    features['Ind'+m+'Blunder'] = data[m+'Scores'].apply(find_erros,args=('ib',))
    for c in color:
        features['Ind'+c+m+'Error'] = data[c+m+'Scores'].apply(find_erros,args=('ie',))
        features['Ind'+c+m+'Blunder'] = data[c+m+'Scores'].apply(find_erros,args=('ib',))
    
#количество общих ошибок и по цветам
features['NumErrors'] = data.Scores.apply(find_erros,args=('ce',))
features['NumBlunders'] = data.Scores.apply(find_erros,args=('cb',))
for c in color:
    features['Num'+c+'Errors'] = data[c+'Scores'].apply(find_erros,args=('ce',))
    features['Num'+c+'Blunders'] = data[c+'Scores'].apply(find_erros,args=('cb',))
    
# количество общих ошибок и по цветам в разных частях партии
for m in modes:
    features['Num'+m+'Errors'] = data[m+'Scores'].apply(find_erros,args=('ce',))
    features['Num'+m+'Blunders'] = data[m+'Scores'].apply(find_erros,args=('cb',))
    for c in color:
        features['Num'+c+m+'Errors'] = data[c+m+'Scores'].apply(find_erros,args=('ce',))
        features['Num'+c+m+'Blunders'] = data[c+m+'Scores'].apply(find_erros,args=('cb',))
        
print(features.shape)
features.head()

(24255, 200)


,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,NumWhiteMitErrors,NumWhiteMitBlunders,NumBlackMitErrors,NumBlackMitBlunders,NumEndErrors,NumEndBlunders,NumWhiteEndErrors,NumWhiteEndBlunders,NumBlackEndErrors,NumBlackEndBlunders
0,0.5,19,23.947368,3,54,38,-26,73,20.5,26.431587,...,1,0,0,0,0,0,0,0,0,0
1,0.0,53,-692.198113,18,-11544,106,-11544,93,63.0,2286.131735,...,0,0,0,0,17,11,7,7,9,8
2,1.0,39,49.246753,31,196,77,-30,198,37.0,51.525093,...,1,0,2,0,1,0,2,0,0,0
3,1.0,25,306.040816,18,875,49,10,917,142.0,313.263595,...,1,1,5,1,5,1,1,1,2,1
4,0.5,29,30.827586,18,-11,58,-15,94,29.0,24.845827,...,0,0,0,0,1,0,2,0,1,0


## Подобие ECO (Encyclopedia of Chess Openings)
##### (не нашла, где подгрузить базу)
Определяем первые три хода как категориальную переменную.

Если в этом дебюте:
     >50 партий - 'частый'
     <50, но >10 партий - 'обычный'
     < 10  партий - 'редкий'

In [13]:
data['ECO']=data.Moves.apply(lambda x: x[:6])

ecos={}
cnt=0
for moves in data['ECO']:
    str_moves="".join(map(str, moves))
    if str_moves not in ecos:
        ecos[str_moves]=1
    else:
        ecos[str_moves]+=1

In [14]:
def get_eco(x,mode='fr'):
    moves ="".join(map(str, x))
    if ecos[moves] > 50 and mode =='fr':
        return 1
    elif ecos[moves] <= 50 and ecos[moves] > 10 and mode=='c':
        return 1
    elif ecos[moves] <= 10 and mode=='r':
        return 1
    else:
        return 0

features['FrequentDebut']=data.ECO.apply(get_eco,args=('fr',))
features['CommonDebut']=data.ECO.apply(get_eco,args=('c',))
features['RareDebut']=data.ECO.apply(get_eco,args=('r',))       

print(features.shape)
features.head()

(24255, 203)


,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,NumBlackMitBlunders,NumEndErrors,NumEndBlunders,NumWhiteEndErrors,NumWhiteEndBlunders,NumBlackEndErrors,NumBlackEndBlunders,FrequentDebut,CommonDebut,RareDebut
0,0.5,19,23.947368,3,54,38,-26,73,20.5,26.431587,...,0,0,0,0,0,0,0,0,0,1
1,0.0,53,-692.198113,18,-11544,106,-11544,93,63.0,2286.131735,...,0,17,11,7,7,9,8,0,1,0
2,1.0,39,49.246753,31,196,77,-30,198,37.0,51.525093,...,0,1,0,2,0,0,0,0,0,1
3,1.0,25,306.040816,18,875,49,10,917,142.0,313.263595,...,1,5,1,1,1,2,1,0,0,1
4,0.5,29,30.827586,18,-11,58,-15,94,29.0,24.845827,...,0,1,0,2,0,1,0,0,1,0


## Признаки, извлеченные из игр
##### (может обновляться)

In [15]:
from chess import QUEEN,PAWN,KNIGHT,BISHOP,ROOK,KING, WHITE,BLACK
#(PAWN = 1, KNIGHT = 2, BISHOP = 3,ROOK = 4, QUEEN = 5, KING = 6)

def extract_game_features(uci):
    game = pgn.Game()
    node = game.add_variation(chess.Move.from_uci(uci[0]))
    for move in uci[1:]:
        node = node.add_variation(chess.Move.from_uci(move))

    first_check = True
    first_queen_move = True
    first_king_move = True
    
    game_features = defaultdict(int)
    
    # ценность фигур на доске (полный набор одного цвета равен 37)
    count_white_pieces=37
    count_black_pieces=37 
    
    node = game
    while node.variations:
        move = node.variation(0).move

        board = node.board()

        # какая фигура пошла на этом ходу и что съела
        moved_piece = board.piece_type_at(move.from_square)
        captured_piece = board.piece_type_at(move.to_square)
        
        if captured_piece is not None:
            if board.turn==WHITE:
                count_black_pieces-=captured_piece
            else:
                count_white_pieces-=captured_piece
    
        white_advantage=count_white_pieces-count_black_pieces if count_white_pieces>count_black_pieces else 0
        black_advantage=count_black_pieces-count_white_pieces if count_black_pieces>count_white_pieces else 0
    
        # количество ходов, сколько держалось материальное преимщущество (по цветам)
        if white_advantage !=0:
            game_features['MovesWhiteAdvantage']+=1
        if black_advantage !=0:
            game_features['MovesBlackAdvantage']+=1 
        
                
        # рокировка (на каком ходу)
        if board.is_kingside_castling(move) and board.turn== WHITE:
            game_features['WhiteKingCastle'] = board.fullmove_number
        elif board.is_queenside_castling(move) and board.turn== WHITE:
            game_features['WhiteQueenCastle'] = board.fullmove_number
        elif board.is_kingside_castling(move) and board.turn== BLACK:
            game_features['BlackKingCastle'] = board.fullmove_number
        elif board.is_queenside_castling(move) and board.turn== BLACK:
            game_features['BlackQueenCastle'] = board.fullmove_number
            
        # первый ход ферзя
        if moved_piece == QUEEN and first_queen_move:
            game_features['QueenMoved'] = board.fullmove_number
            first_queen_move = False
          
        # если съели ферзя (на каком ходу)
        if captured_piece == QUEEN:
            game_features['QuennCaptured'] = board.fullmove_number
            
        # первый ход короля (без учета рокировки)
        if moved_piece == KING and first_king_move:
            game_features['KingMoved'] = board.fullmove_number
            first_king_move = False
            
        # превращение пешки (сколько пешек превратилось)
        if move.promotion: 
            game_features['Promotion'] += 1
         # количество шахов в партии по цветам  
        if board.is_check():
            if board.turn==WHITE:
                game_features['WhiteСhecks'] += 1
            else:
                game_features['BlackСhecks'] += 1
            if first_check:
                game_features['FirstCheck'] = board.fullmove_number
                first_check = False
            
        # количество взятий на проходе
        if board.is_en_passant(move): 
            game_features['EnPassant'] += 1
            
        node = node.variation(0)
        
    # анализ итоговой позиции 
    if board.is_checkmate(): #шах
        game_features['IsCheckmate'] += 1
    if board.is_stalemate(): # мат 
        game_features['IsStalemate'] += 1
    # проверяет, есть ли у одной из сторон достаточно материала для победы (false, если, например, один конь остался)
    if board.is_insufficient_material():
        game_features['InsufficientMaterial'] += 1
    # Проверка на правило 50-ти ходов или троекратное повторение
    if board.can_claim_draw():
        game_features['CanClaimDraw'] += 1

    # фигуры в итоговой позиции
    piece_placement = board.fen().split()[0]
    end_pieces = Counter(x for x in piece_placement if x.isalpha())
    
    # Оценка оставшихся фигур в итоговой позиции
    game_features['WhiteAdvantage']=count_white_pieces-count_black_pieces if count_white_pieces>count_black_pieces else 0
    game_features['BlackAdvantage']=count_black_pieces-count_white_pieces if count_black_pieces>count_white_pieces else 0

    # количество фигур в итоговой позиции
    game_features.update({'End' + piece: cnt
                     for piece, cnt in end_pieces.items()})
    
    return game_features

all_features = []
for uci in data.UCI:
    ext_features = extract_game_features(uci)
    all_features.append(ext_features)

In [ ]:
game_features['MovesWhiteAdvantage']=np.NaN
game_features['MovesBlackAdvantage']=np.Nan
game_features['WhiteKingCastle'] = np.NaN
game_features['WhiteQueenCastle'] = np.NaN
game_features['BlackKingCastle'] = np.NaN
game_features['BlackQueenCastle'] = np.NaN
game_features['QueenMoved'] =np.NaN
game_features['QuennCaptured'] =np.NaN
game_features['KingMoved'] = np.NaN
game_features['Promotion'] =np.NaN
game_features['WhiteСhecks'] =np.NaN
game_features['BlackСhecks'] =np.NaN
game_features['FirstCheck'] = board.fullmove_number
game_features['EnPassant'] += 1
game_features['IsCheckmate'] += 1
game_features['IsStalemate'] += 1
game_features['InsufficientMaterial'] += 1
game_features['CanClaimDraw'] += 1
game_features['WhiteAdvantage']=count_white_pieces-count_black_pieces if count_white_pieces>count_black_pieces else 0
game_features['BlackAdvantage']=count_black_pieces-count_white_pieces if count_black_pieces>count_white_pieces else 0
game_features['Endr']
game_features['Endk']
game_features['Endp']
game_features['Endb']
game_features['Endn']
game_features['Endq']
game_features['EndN']
game_features['EndP']
game_features['EndQ']
game_features['EndB']
game_features['EndR']
game_features['EndK']



In [16]:
items = all_features[0].items()
columns=[i[0] for i in items]
df_features=pd.DataFrame(columns=columns)

for ind,feature in enumerate(all_features):   
    for key, value in feature.items():
        df_features.loc[ind,key]=value

In [17]:
print(len(columns))
print(df_features.shape)
df_features.head()

23
(24255, 30)


,BlackKingCastle,KingMoved,WhiteKingCastle,QueenMoved,MovesWhiteAdvantage,MovesBlackAdvantage,BlackСhecks,FirstCheck,QuennCaptured,WhiteAdvantage,...,EndB,EndR,EndK,WhiteСhecks,WhiteQueenCastle,BlackQueenCastle,Promotion,EnPassant,CanClaimDraw,InsufficientMaterial
0,5.0,5.0,7.0,9.0,11.0,2.0,1.0,14.0,19.0,0.0,...,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.0,7.0,8.0,13.0,9.0,56.0,4.0,13.0,16.0,0.0,...,NaN,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7.0,7.0,NaN,7.0,4.0,43.0,4.0,24.0,22.0,0.0,...,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.0,8.0,5.0,31.0,1.0,4.0,15.0,NaN,7.0,...,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,6.0,12.0,8.0,8.0,4.0,NaN,NaN,NaN,0.0,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
features=pd.concat([features, df_features], axis=1)
features=features.fillna(0)
features.shape

(24255, 233)

In [19]:
corr=features.corr(method='pearson')
corr=corr.fillna(0)
corr.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,EndB,EndR,EndK,WhiteСhecks,WhiteQueenCastle,BlackQueenCastle,Promotion,EnPassant,CanClaimDraw,InsufficientMaterial
Result,1.000000,-0.032734,0.680422,0.154037,0.567336,-0.043769,0.436827,0.407523,0.087790,0.002583,...,0.062679,0.068787,0.0,-0.145652,0.022463,-0.001100,-0.008031,0.014029,-0.019784,-0.003008
NumMoves,-0.032734,1.000000,-0.016992,-0.013831,0.000671,0.999882,-0.194297,0.191654,0.146977,0.252136,...,-0.375284,-0.501649,0.0,0.561510,-0.030204,-0.008063,0.241533,0.045769,0.015195,0.053027
MeanScore,0.680422,-0.016992,1.000000,0.381663,0.751580,-0.024856,0.631160,0.646303,0.116982,0.052224,...,0.066542,0.050717,0.0,-0.133252,0.022777,0.004563,0.001572,0.017114,-0.012355,-0.002614
ModeScore,0.154037,-0.013831,0.381663,1.000000,0.190907,-0.015617,0.192575,0.167771,0.018366,-0.009792,...,0.034899,0.023327,0.0,-0.053992,0.002986,-0.002107,-0.008549,-0.006883,-0.004083,-0.000749
FinalScore,0.567336,0.000671,0.751580,0.190907,1.000000,-0.005774,0.628106,0.644216,0.068602,0.047178,...,0.037475,0.027011,0.0,-0.081362,0.011280,-0.000963,-0.007771,0.005521,-0.007661,-0.001356


In [25]:
indexes=corr.index
corr_feat=[]
for index in corr.index:
    for column in corr.index:
        if index != column and abs(corr.loc[index,column])>=0.7:
            corr_feat.append((index,column))
            
corr_feat[:5], len(corr_feat)

([('NumMoves', 'FullMoves'),
  ('MeanScore', 'FinalScore'),
  ('MeanScore', 'MeanWhiteScore'),
  ('MeanScore', 'FinalWhiteScore'),
  ('MeanScore', 'MeanBlackScore')],
 1342)

In [26]:
features.to_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv', sep=',',index=False)

#### Подумать над тем, чтобы добавить веса наиболее важным признакам

(Длинные игры вероятно более важные для понимания мастерства игры, поэтому умножение таких функций, как дисперсия и среднее, на количество ходов может быть полезно)